In [2]:
#import libraries and functions
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

#read train and test data
X_full = pd.read_csv('~/Desktop/Kaggle/practice/Intro_Machine_Learning/House_Price_Competition/train.csv', index_col='Id')
X_test_full = pd.read_csv('~/Desktop/Kaggle/practice/Intro_Machine_Learning/House_Price_Competition/test.csv', index_col='Id')

#remove rows with missing target, and separate target with predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

#set predictor
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

#break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

### Drop columns with missing values

In [4]:
#function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [6]:
#get name of the columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

#drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [8]:
print('MAE:', score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE: 17837.82570776256


### Imputation